In [224]:
from pyspark.sql.functions import *

off_train = spark.read.csv('ccf_offline_stage1_train.csv', header=True, inferSchema=True)
off_test = spark.read.csv('ccf_offline_stage1_test_revised.csv', header = True, inferSchema=True)
on_train = spark.read.csv('ccf_online_stage1_train.csv', header = True, inferSchema = True)

In [445]:
dataset3 = off_test
feature3 = off_train.where(((off_train.Date>='20160315') & (off_train.Date<='20160630')) \
                                   | ((off_train.Date=='null' )& (off_train.Date_received>='20160315') & (off_train.Date_received<='20160630')))\
                              .select('*')
dataset2 =  off_train.where((off_train.Date_received>='20160515')&(off_train.Date_received<='20160615'))\
                    .select('*')
feature2 = off_train.where((off_train.Date>='20160201')&(off_train.Date<='20160514')\
                           |((off_train.Date=='null')&(off_train.Date_received>='20160201')&(off_train.Date_received<='20160514')))\
                    .select('*')
dataset1 = off_train.where((off_train.Date_received>='20160414')&(off_train.Date_received<='20160514'))\
                    .select('*')
feature1 = off_train.where((off_train.Date>='20160101')&(off_train.Date<='20160413')\
                           |((off_train.Date=='null')&(off_train.Date_received>='20160101')&(off_train.Date_received<='20160413')))\
                    .select('*')

############# coupon related feature   #############
"""
coupon related: 
      discount_rate. discount_man. discount_jian. is_man_jian
      day_of_week,day_of_month. (date_received)

In [446]:
def calc_discount_rate(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return float(s[0])
    else:
        return 1.0-float(s[1])/float(s[0])

def get_discount_man(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return 'null'
    else:
        return int(s[0])
        
def get_discount_jian(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return 'null'
    else:
        return int(s[1])

def is_man_jian(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return 0
    else:
        return 1

In [447]:
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
# spark.udf.register('calc_discount_rate', calc_discount_rate, FloatType())
calc_discount_rate = udf(calc_discount_rate, FloatType())
get_discount_man = udf(get_discount_man)
get_discount_jian = udf(get_discount_jian, IntegerType())
is_man_jian = udf(is_man_jian, IntegerType())

for dataset3

In [448]:
#dataset3['day_of_week'] = test1.apply(lambda x:int(x[6:8]))
#spark.udf.register('week_day', lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
from datetime import date
week_day = udf(lambda x:x if(x == 'null') else date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
month_day = udf(lambda x:x if(x == 'null') else int(x[6:8]))
dayDistance = udf(lambda x:x if(x == 'null') else (date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,6,30)).days)

In [449]:
dataset3 = dataset3.withColumn('Day_of_week', week_day(col('Date_received').astype('string')))
dataset3 = dataset3.withColumn('Day_of_mouth', month_day(col('Date_received').astype('string')))
dataset3 = dataset3.withColumn('Days_distance', dayDistance(col('Date_received').astype('string')))

In [450]:
dataset3.withColumn('Discount_man', get_discount_man(col('Discount_rate')))
dataset3.withColumn('Discount_jian', get_discount_jian(col('Discount_rate')))
dataset3.withColumn('Is_man_jian', is_man_jian(col('Discount_rate')))
dataset3.withColumn('Discount_rate', calc_discount_rate(col('Discount_rate')))

DataFrame[User_id: int, Merchant_id: int, Coupon_id: int, Discount_rate: float, Distance: string, Date_received: int, Day_of_week: string, Day_of_mouth: string, Days_distance: string]

In [451]:
d = dataset3.select('Coupon_id')
# 赋值 d.withColumn('col', lit(1))
d = d.groupBy('Coupon_id').count().withColumnRenamed('count', 'Coupon_count')

In [452]:
dataset3 = dataset3.join(d, 'Coupon_id')
# dataset3.reorderColumns(seq('User_id', 'Merchant_id', 'Coupon_id', 'Coupon_count', 'Distance', 'Date_received', 'day_of_week', 'day_of_mouth', 'days_distance'))
dataset3.write.csv('coupon3_feature', header = True)

for dataset2

In [453]:
dataset2 = dataset2.withColumn('Day_of_week', week_day(col('Date_received').astype('string')))
dataset2 = dataset2.withColumn('Day_of_mouth', month_day(col('Date_received').astype('string')))
dataset2 = dataset2.withColumn('Days_distance', dayDistance(col('Date_received').astype('string')))
dataset2.withColumn('Discount_man', get_discount_man(col('Discount_rate')))
dataset2.withColumn('Discount_jian', get_discount_jian(col('Discount_rate')))
dataset2.withColumn('Is_man_jian', is_man_jian(col('Discount_rate')))
dataset2.withColumn('Discount_rate', calc_discount_rate(col('Discount_rate')))

d = dataset2.select('Coupon_id')
d = d.groupBy('Coupon_id').count().withColumnRenamed('count', 'Coupon_count')

dataset2 = dataset2.join(d, 'Coupon_id')
dataset2.write.csv('coupon2_feature', header = True)

for dataset1

In [454]:
dataset1 = dataset1.withColumn('Day_of_week', week_day(col('Date_received').astype('string')))
dataset1 = dataset1.withColumn('Day_of_mouth', month_day(col('Date_received').astype('string')))
dataset1 = dataset1.withColumn('Days_distance', dayDistance(col('Date_received').astype('string')))
dataset1.withColumn('Discount_man', get_discount_man(col('Discount_rate')))
dataset1.withColumn('Discount_jian', get_discount_jian(col('Discount_rate')))
dataset1.withColumn('Is_man_jian', is_man_jian(col('Discount_rate')))
dataset1.withColumn('Discount_rate', calc_discount_rate(col('Discount_rate')))

d = dataset1.select('Coupon_id')
d = d.groupBy('Coupon_id').count().withColumnRenamed('count', 'Coupon_count')

dataset1 = dataset1.join(d, 'Coupon_id')
dataset1.write.csv('coupon1_feature', header = True)

############# merchant related feature   #############
"""merchant related: 
      total_sales. sales_use_coupon.  total_coupon
      coupon_rate = sales_use_coupon/total_sales.  
      transfer_rate = sales_use_coupon/total_coupon. 
      merchant_avg_distance,merchant_min_distance,merchant_max_distance of those use coupon
"""

In [391]:
merchant3 = feature3.select('Merchant_id', 'Coupon_id', 'Distance', 'Date_received', 'Date')

In [392]:
null_ex = udf(lambda x: 0 if(x == 'null') else x)
t = merchant3.select('Merchant_id').distinct()
t1 = merchant3.where(merchant3.Date!='null')\
              .select('Merchant_id')\
              .groupby('Merchant_id').count()\
              .withColumnRenamed('count', 'Total_sales')

In [393]:
t2 = merchant3.where((merchant3.Date != 'null') & (merchant3.Coupon_id != 'null'))\
              .select('Merchant_id')\
              .groupby('Merchant_id').count()\
              .withColumnRenamed('count', 'Sales_use_coupon')\
              .withColumn('Sales_use_coupon', null_ex(col('Sales_use_coupon')))

In [394]:
t3 = merchant3.where(merchant3.Coupon_id != 'null')\
              .select('Merchant_id')\
              .groupby('Merchant_id').count()\
              .withColumnRenamed('count', 'Total_coupon')\
              .withColumn('Total_coupon', null_ex(col('Total_coupon')))

数据转换在select中进行才有效（不知道为什么）

In [395]:
extract_null = udf(lambda x: -1 if(x == 'null') else x)
t4 = merchant3.where((merchant3.Date!='null')&(merchant3.Coupon_id!='null'))\
              .select('Merchant_id',extract_null('Distance').cast(IntegerType()))\
              .withColumnRenamed('CAST(<lambda>(Distance) AS INT)', 'Distance')

In [402]:
t5 = t4.groupby('Merchant_id').min('Distance')\
       .withColumnRenamed('min(Distance)', 'Merchant_min_distance')

In [403]:
t6 = t4.groupby('Merchant_id').max('Distance')\
       .withColumnRenamed('max(Distance)', 'Merchant_max_distance')
t6

DataFrame[Merchant_id: int, Merchant_max_distance: int]

In [408]:
t7 = t4.groupby('Merchant_id').mean('Distance')\
       .withColumnRenamed('avg(Distance)', 'Merchant_mean_distance')

In [409]:
med = expr('percentile_approx(Distance, 0.5)')
t8 = t4.groupby('Merchant_id').agg(med.alias('Merchant_median_distance'))

In [410]:
merchant3_feature = t.join(t1, 'Merchant_id')\
                     .join(t2, 'Merchant_id')\
                     .join(t3, 'Merchant_id')\
                     .join(t5, 'Merchant_id')\
                     .join(t6, 'Merchant_id')\
                     .join(t7, 'Merchant_id')\
                     .join(t8, 'Merchant_id')
merchant3_feature

DataFrame[Merchant_id: int, Total_sales: bigint, Sales_use_coupon: string, Total_coupon: string, Merchant_min_distance: int, Merchant_max_distance: int, Merchant_mean_distance: double, Merchant_median_distance: int]

In [411]:
merchant3_feature = merchant3_feature.withColumn('Merchant_coupon_transfer_rate', (col('Sales_use_coupon')\
                 .cast(FloatType())) / merchant3_feature.Total_coupon)

In [412]:
merchant3_feature = merchant3_feature.withColumn('Coupon_rate', (col('Sales_use_coupon').cast(FloatType()))\
                                                 /  merchant3_feature.Total_sales)

In [413]:
merchant3_feature.repartition(1).write.csv('merchant3_feature', header = True)

for dataset2

In [455]:
merchant2 = feature2.select('Merchant_id', 'Coupon_id', 'Distance', 'Date_received', 'Date')
null_ex = udf(lambda x: 0 if(x == 'null') else x)
t = merchant2.select('Merchant_id').distinct()
t1 = merchant2.where(merchant2.Date!='null')\
              .select('Merchant_id')\
              .groupby('Merchant_id').count()\
              .withColumnRenamed('count', 'Total_sales')
t2 = merchant2.where((merchant2.Date != 'null') & (merchant2.Coupon_id != 'null'))\
              .select('Merchant_id')\
              .groupby('Merchant_id').count()\
              .withColumnRenamed('count', 'Sales_use_coupon')\
              .withColumn('Sales_use_coupon', null_ex(col('Sales_use_coupon')))
t3 = merchant2.where(merchant2.Coupon_id != 'null')\
              .select('Merchant_id')\
              .groupby('Merchant_id').count()\
              .withColumnRenamed('count', 'Total_coupon')\
              .withColumn('Total_coupon', null_ex(col('Total_coupon')))
extract_null = udf(lambda x: -1 if(x == 'null') else x)
t4 = merchant2.where((merchant2.Date!='null')&(merchant2.Coupon_id!='null'))\
              .select('Merchant_id',extract_null('Distance').cast(IntegerType()))\
              .withColumnRenamed('CAST(<lambda>(Distance) AS INT)', 'Distance')
t5 = t4.groupby('Merchant_id').min('Distance')\
       .withColumnRenamed('min(Distance)', 'Merchant_min_distance')
t6 = t4.groupby('Merchant_id').max('Distance')\
       .withColumnRenamed('max(Distance)', 'Merchant_max_distance')
t7 = t4.groupby('Merchant_id').mean('Distance')\
       .withColumnRenamed('avg(Distance)', 'Merchant_mean_distance')
med = expr('percentile_approx(Distance, 0.5)')
t8 = t4.groupby('Merchant_id').agg(med.alias('Merchant_median_distance'))
merchant2_feature = t.join(t1, 'Merchant_id')\
                     .join(t2, 'Merchant_id')\
                     .join(t3, 'Merchant_id')\
                     .join(t5, 'Merchant_id')\
                     .join(t6, 'Merchant_id')\
                     .join(t7, 'Merchant_id')\
                     .join(t8, 'Merchant_id')
merchant2_feature = merchant2_feature.withColumn('Merchant_coupon_transfer_rate', (col('Sales_use_coupon')\
                 .cast(FloatType())) / merchant2_feature.Total_coupon)
merchant2_feature = merchant2_feature.withColumn('Coupon_rate', (col('Sales_use_coupon').cast(FloatType()))\
                                                 /  merchant2_feature.Total_sales)
merchant2_feature.repartition(1).write.csv('merchant2_feature', header = True)

for dataset1

In [456]:
merchant1 = feature1.select('Merchant_id', 'Coupon_id', 'Distance', 'Date_received', 'Date')
null_ex = udf(lambda x: 0 if(x == 'null') else x)
t = merchant1.select('Merchant_id').distinct()
t1 = merchant1.where(merchant1.Date!='null')\
              .select('Merchant_id')\
              .groupby('Merchant_id').count()\
              .withColumnRenamed('count', 'Total_sales')
t2 = merchant1.where((merchant1.Date != 'null') & (merchant1.Coupon_id != 'null'))\
              .select('Merchant_id')\
              .groupby('Merchant_id').count()\
              .withColumnRenamed('count', 'Sales_use_coupon')\
              .withColumn('Sales_use_coupon', null_ex(col('Sales_use_coupon')))
t3 = merchant1.where(merchant1.Coupon_id != 'null')\
              .select('Merchant_id')\
              .groupby('Merchant_id').count()\
              .withColumnRenamed('count', 'Total_coupon')\
              .withColumn('Total_coupon', null_ex(col('Total_coupon')))
extract_null = udf(lambda x: -1 if(x == 'null') else x)
t4 = merchant1.where((merchant1.Date!='null')&(merchant1.Coupon_id!='null'))\
              .select('Merchant_id',extract_null('Distance').cast(IntegerType()))\
              .withColumnRenamed('CAST(<lambda>(Distance) AS INT)', 'Distance')
t5 = t4.groupby('Merchant_id').min('Distance')\
       .withColumnRenamed('min(Distance)', 'Merchant_min_distance')
t6 = t4.groupby('Merchant_id').max('Distance')\
       .withColumnRenamed('max(Distance)', 'Merchant_max_distance')
t7 = t4.groupby('Merchant_id').mean('Distance')\
       .withColumnRenamed('avg(Distance)', 'Merchant_mean_distance')
med = expr('percentile_approx(Distance, 0.5)')
t8 = t4.groupby('Merchant_id').agg(med.alias('Merchant_median_distance'))
merchant1_feature = t.join(t1, 'Merchant_id')\
                     .join(t2, 'Merchant_id')\
                     .join(t3, 'Merchant_id')\
                     .join(t5, 'Merchant_id')\
                     .join(t6, 'Merchant_id')\
                     .join(t7, 'Merchant_id')\
                     .join(t8, 'Merchant_id')
merchant1_feature = merchant1_feature.withColumn('Merchant_coupon_transfer_rate', (col('Sales_use_coupon')\
                 .cast(FloatType())) / merchant1_feature.Total_coupon)
merchant1_feature = merchant1_feature.withColumn('Coupon_rate', (col('Sales_use_coupon').cast(FloatType()))\
                                                 /  merchant1_feature.Total_sales)
merchant1_feature.repartition(1).write.csv('merchant1_feature', header = True)